<a href="https://colab.research.google.com/github/nspiegeln/iml_2022/blob/main/task4_main13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""task4_main8.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1uPxuWvUseQEoYNbB9dtpwKwMsNZCjk-9
"""

import pandas as pd
import torch
from torch import nn
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses, metrics
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

#load data
from google.colab import drive
drive.mount('/content/gdrive')

path = "gdrive/MyDrive/task4/"
df_train= pd.read_csv(path + "train_features.csv")
df_train_labels= pd.read_csv(path + "train_labels.csv")
df_pretrain= pd.read_csv(path +"pretrain_features.csv")
df_pretrain_labels= pd.read_csv(path +"pretrain_labels.csv")
df_test= pd.read_csv(path +"test_features.csv")

df_train_values = df_train.iloc[:,2::]
df_pretrain_values = df_pretrain.iloc[:,2::]

df_train_lab = df_train_labels.iloc[:,1]
df_pretrain_lab = df_pretrain_labels.iloc[:,1]

train = df_train_values.values #dataset with known homo-lumo gap
pretrain = df_pretrain_values.values #dataset with known lumo value

train_labels = df_train_lab.values
pretrain_labels = df_pretrain_lab.values

x_pretrain, x_pretest, y_pretrain, y_pretest = train_test_split(pretrain, pretrain_labels, test_size = 0.1)
print(x_pretrain.shape[0], x_pretrain.shape[1])
print(x_pretest.shape[0], x_pretest.shape[1])

#get device
def get_default_device():
    """Picking GPU if available or else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

device = get_default_device()

# PRE-PROCESSING of features according to autoencoder
# pre-train set

#hyperparameters
dim = 200
epochs = 100         #resuls still decrease after epoch 100, @epoch=100 loss=3.4e-3
batch_size = 64

# create encoder and decoder to reduce dimensionality of features
class Autoencoder(Model):
  def __init__(self, dim):
    super(Autoencoder, self).__init__()
    self.dim = dim
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(500, activation='relu'),
      layers.Dense(200, activation='relu'),
    ])
    self.trainor = tf.keras.Sequential([
      layers.Dense(100, activation='relu'),
      
    ])
    self.predictor = tf.keras.Sequential([
      layers.Dense(100, activation='relu'),
      layers.Dense(50, activation='relu'),
      layers.Dense(1),
    ])                                     

  def call(self, x):
    encoded = self.encoder(x)
    trained = self.trainor(encoded)
    predicted = self.predictor(trained)
    return predicted

autoencoder = Autoencoder(dim)
sgd = SGD(learning_rate=0.1, decay=0.9, momentum=0.9) #changed decay from 1e-2 to 0.9 loss = 3.5e-3
autoencoder.compile(optimizer='sgd', loss=losses.MeanSquaredError())

autoencoder.fit(x_pretrain, y_pretrain, epochs=epochs, batch_size = batch_size, shuffle=True, validation_data=(x_pretest, y_pretest))

y_predpretrain = autoencoder.predict(x_pretrain)

# Transfer learning to HOMO-LUMO gap learning
# freeze everything but last layer
for layer in autoencoder.layers[:-1]:
  layer.trainable = False

# autoencoder.encoder.summary()
# autoencoder.trainor.summary()
# autoencoder.predictor.summary()

dim = 1
epochs = 800   #loss decreases to loss=6.8e-3 after 1000 epochs
class AutoencoderNew(Model):
  def __init__(self, dim):
    super(AutoencoderNew, self).__init__()
    self.dim = dim
    self.encoder = tf.keras.Sequential([
      autoencoder,
      layers.Dense(50, activation='relu'),
      layers.Dense(100, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(100, activation='relu'),
      layers.Dense(50, activation='relu'),
      layers.Dense(1),
    ])                                    

  def call(self, x):
    encoded = self.encoder(x)
    return encoded

model = AutoencoderNew(dim)

sgd = SGD(learning_rate=0.1, decay=0.9, momentum=0.9)

model.compile(optimizer='sgd', loss=losses.MeanSquaredError())
model.fit(train, train_labels, epochs=epochs, shuffle=True)

#predict homolumo gap for testset
x_test = df_test.iloc[:,2::]
id_test = df_test.iloc[:,0]

y_test = model.predict(x_test)

#export solution
df_evaltest = pd.DataFrame(y_test, columns = ['y'])
df_evaltest.insert(0, 'Id', id_test)
df_evaltest.to_csv('solutiontot8.csv', index=False, index_label=False, encoding = 'utf-8')